# **IMPORT LIBRERIE**

In [ ]:
import tensorflow as tf 
import numpy as np
import keras
import tensorflow_datasets as tfds

tf.compat.v1.enable_eager_execution()
tf.executing_eagerly() 

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1)
tf.random.set_seed(2)

# **APRE IL FILE CON LABELS E SALVA I VALORI IN UN DIZIONARIO**
Il dizionario ha come chiave il nome del file e valore il valore della label

In [ ]:
import csv

def read_labels(filename,train=True):
  csv_file = open(filename)
  reader = csv.reader(csv_file, delimiter=',', quotechar='|')
  rows = list(reader)
  labels_dict = {}

  y = []
  for row in rows:
    value = int(row[1])
    labels_dict[row[0]]=value-1
    y.append(value)

  if train:
    NUM_CLASSES = np.unique(y).shape[0] 
    return labels_dict,NUM_CLASSES
  else:
    return labels_dict,None
  

train_labels_dict,NUM_CLASSES = read_labels('/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/data/train_labels.csv',train = True)
valid_labels_dict,_ = read_labels('/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/data/valid_labels.csv',train = False)



In [ ]:
# legge il file contentnente i pesi per ogni classe e li salva in un dizionario
csv_file = open('/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/data/weights.csv')
reader = csv.reader(csv_file, delimiter=',', quotechar='|')
rows = list(reader)

class_weights = {}

for i in range(len(rows)):
  class_weights[int(rows[i][0])] = float(rows[i][1])


# **CLASSE FACE DETECTOR**

In [ ]:
modelFile = "/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/data/res10_300x300_ssd_iter_140000_fp16.caffemodel"
configFile = "/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/data/deploy.prototxt"

class FaceDetector:
    net = None
    def __init__(self, min_confidence=0.5):
        print ("FaceDetector -> init")
        self.net = cv.dnn.readNetFromCaffe(configFile, modelFile)
        self.min_confidence = min_confidence
        print ("FaceDetector -> init ok")
    
    def detect(self, image):
        blob = cv.dnn.blobFromImage(image, 1.0, (128, 128), [106, 121, 150], False, False)
        frameHeight, frameWidth, channels = image.shape
        self.net.setInput(blob)
        detections = self.net.forward()
        faces_result=[]
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > self.min_confidence:
                x1 = int(detections[0, 0, i, 3] * frameWidth)
                y1 = int(detections[0, 0, i, 4] * frameHeight)
                x2 = int(detections[0, 0, i, 5] * frameWidth)
                y2 = int(detections[0, 0, i, 6] * frameHeight)
                f = (x1,y1, x2-x1, y2-y1)
                if f[2]>1 and f[3]>1:
                    faces_result.append({
                        'roi': f,
                        'type': 'face',
                        'img': image[f[1]:f[1]+f[3], f[0]:f[0]+f[2]],
                        'confidence' : confidence
                    })
        return faces_result
    
    def __del__(self):
        print ("FaceDetector -> bye")


def findRelevantFace(objs, W,H):
    mindistcenter = None
    minobj = None
    for o in objs:
        cx = o['roi'][0] + (o['roi'][2]/2)
        cy = o['roi'][1] + (o['roi'][3]/2)
        distcenter = (cx-(W/2))**2 + (cy-(H/2))**2
        if mindistcenter is None or distcenter < mindistcenter:
            mindistcenter = distcenter
            minobj = o
    return minobj

# **FUNZIONI PREPROCESSING DATASET**

In [ ]:
from PIL import Image
from IPython.display import display
import dlib
import cv2 as cv
from random import getrandbits,randint
from imutils import rotate



size = (100,100,3)

def show_image(arr,norm=True):
  """Mostra un'immagine

  Parameters
  ----------
  arr : np.ndarray
      Immagine da visualizzare
  norm : boolean, optional
      Booleano che indica se l'immagine che viene passata è stata normalizzata nel range [0,1]

  Returns
  -------
  None
  """
  if norm:
    arr = arr*255
  imdata=arr.astype('uint8')
  imdata=np.flip(imdata,-1)
  img=Image.fromarray(imdata)
  
  display(img)

  

def preprocessing(img,label,resize=True,size=None,augment=False):
  """Effettua il preprocessing dell'immagine

  Parameters
  ----------
  img : np.ndarray
      L'immagine da preprocessare
  label: int
      Label associata all'immagine
  resize: bool, optional
      Se true effettua il resize dell'immagine 
  size: tuple, optional
      Se resize = True, effettua il resize della dimensione specificata

  Returns
  -------
  tf.Tensor
      L'immagine preprocessata sotto forma di tensore
  """
  if resize:
    img = cv.resize(img,(size,size))
  
  #applica il clahe
  lab = cv.cvtColor(img, cv.COLOR_BGR2LAB)
  lab_planes = cv.split(lab)
  lab_planes[0] = clahe.apply(lab_planes[0])
  lab = cv.merge(lab_planes)
  img = cv.cvtColor(lab, cv.COLOR_LAB2BGR)
  if augment:
    img = augmentation(img)
  else:
    img = img.astype(np.float32)
    img = img/255.0
    img = tf.convert_to_tensor(img,dtype=tf.float32)
  return img


def augmentation(img):
  """Applica la data augmentation sull'immagine 

  Parameters
  ----------
  img : np.ndarray
        immagine su cui applicare la data augmentation

  Returns
  tf.Tensor
    Tensore che contiene l'immagine
  """
    #random flip orizzontale e verticale
  if bool(getrandbits(1)):
    img = cv.flip(img,1)

  image = tf.convert_to_tensor(img,tf.float32)
  #random brightness
  image = tf.image.random_brightness(image,max_delta=0.5)
  #random contrast
  image = tf.image.random_contrast(image,lower=0.1,upper=1)
  #random saturation
  image = tf.image.random_saturation(image, lower=0.1,upper=1)
  #normalizza nel range [0,1]
  image = tf.cast(image,tf.float32)/255.0

  return image


def align_and_preprocess(img,label,augment=True):
  """
     Chiama la funzione detect and align utilizzando la funzione wrapper di tensorflow py_function

  Parameters
  ----------
  img : ndarray
        Immagine da preprocessare
  label : float
        label associata all'imamgine
  augment : bool optional
        se true, applica la data augmentation

  Returns
  tuple
    Tupla che contiene l'immagine e la label processate
  """
  image = tf.py_function(detect_and_align,[img,label,size,augment],tf.float32)

  #if train:
  #  labels_dict = train_labels_dict
  #else:
  #  labels_dict = valid_labels_dict 

  #label = tf.py_function(map_labels,[sample['file_name'],sample['label'],labels_dict],tf.float32)
  #label = tf.reshape(label, [NUM_CLASSES])
  return image, label


def map_labels(filename):
  """Effettua il mapping tra un campione del dataset e la label necessaria

  Parameters
  ----------
  filename : tf.Tensor
        Tensore che contiene il nome del file
  label : tf.Tensor
        Tensore che contine la label originaria del dataset

  Returns
  float
    Label da utilizzare. Se non esiste un mapping filename labels_dict ritorna 255.0
  """

  filename = filename.numpy().decode('UTF-8')
  if filename in labels_dict:
    return tf.keras.utils.to_categorical(labels_dict[filename],NUM_CLASSES)
  else:
    return tf.fill([NUM_CLASSES],255.0)
  
@tf.function
def filter_labels(img,label):
  """Filtra dal dataset tutti i campioni a cui non è associata una label. I file che non hanno label sono mappati con la label 255.0

  Parameters
  ----------
  filename : tf.Tensor
        Tensore che contiene l'immagine
  label : tf.Tensor
        Tensore che contine la label

  Returns
  bool
    True se label != 255, False altrimenti
  """
  label = tf.cast(label,tf.uint8)

  a = mask_label.__eq__(label)
  a = tf.reduce_all(a)
  return not a

def findRelevantFace(objs, W,H):
  """Trova il volto principale tra i volti trovati nella face detection
  Parameters
  ----------
  objs : tf.Tensor
        lista di dizionari, ogni elemento della lista contiene informazioni su un volto rilevato
  W : tf.Tensor
        larghezza dell'immagine originale
  H : int
        lunghezza dell'immagine originale
  Returns
  bool
    True se label != 255, False altrimenti

  """
  mindistcenter = None
  minobj = None
  for o in objs:
      cx = o['roi'][0] + (o['roi'][2]/2)
      cy = o['roi'][1] + (o['roi'][3]/2)
      distcenter = (cx-(W/2))**2 + (cy-(H/2))**2
      if mindistcenter is None or distcenter < mindistcenter:
          mindistcenter = distcenter
          minobj = o
  return minobj

def detect_and_align(img,label,size,augment):
  """Effettua la detection della faccia nell'immagine e allinea il volto.

  Parameters
  ----------
  img : tf.Tensor
        Tensore che contiene l'immagine
  label : tf.Tensor
        Tensore che contine la label
  size : tuple
        Dimensione desiderata dell'immagine preprocessata
  augment : bool
        Se true, applica la data augmentation
  Returns
  Tf.Tensor
    Tensore che contine l'immagine preprocessata
  """
  resize= False
  img_array = img.numpy()
  #img_array = img.copy()
  detected_imgs = face_det.detect(img_array)
  if len(detected_imgs) != 0:
      relevant = findRelevantFace(detected_imgs,img_array.shape[1],img_array.shape[0])
      roi = relevant['roi']
      rect = dlib.rectangle(roi[0],roi[1],roi[0]+roi[2],roi[1]+roi[3])
      face = sp(img_array, rect)
      img_array = dlib.get_face_chip(img_array, face, size=size[0])
  else:
      resize = True
  return preprocessing(img_array,label,resize,size[0],augment)


def preprocess_labels(sample):
  """Funzione che effettua il preprocessing delle labels
  chiama la funzione map labels che associa al campione la corrispondente label presente nel file csv
  Parameters
  ----------
  sample: dict
    dizionario contenente il campione su cui effettuare il preproscessing della label
  Returns
  tuple
    tupla che contiene l'immagine e la label preprocessata

  """

  label = tf.py_function(map_labels,[sample['file_name']],tf.float32)
  label = tf.reshape(label, [NUM_CLASSES])
  #sample['label'] = label

  return sample['image'],label



  
clahe = cv.createCLAHE(2,(3,3))
face_det = FaceDetector()
predictor_path = "/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/data/shape_predictor_68_face_landmarks.dat"
sp = dlib.shape_predictor(predictor_path)

mask_label = tf.fill([NUM_CLASSES],255)
mask_label = tf.cast(mask_label,tf.uint8)


# **APRE IL DATASET, SI EFFETTUA IL PREPROCESSING E SPLIT TRAIN VALIDATION**

In [ ]:
from random import randint 

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

ds = tfds.load(
    name='vgg_face2',
    split='train',
    data_dir='/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/vggface2',
    shuffle_files = True,
    as_supervised = False
)
LENGTH = len(ds)

labels_dict = train_labels_dict



train_ds = ds
train_ds = train_ds.map(preprocess_labels,num_parallel_calls=AUTOTUNE)
train_ds = train_ds.filter(filter_labels)
train_ds = train_ds.map(align_and_preprocess,num_parallel_calls=1)
train_ds = train_ds.batch(BATCH_SIZE).cache('/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/train').prefetch(AUTOTUNE)

labels_dict = valid_labels_dict
valid_ds = ds

valid_ds = valid_ds.map(preprocess_labels,num_parallel_calls=AUTOTUNE)
valid_ds = valid_ds.filter(filter_labels)
valid_ds = valid_ds.map(align_and_preprocess,num_parallel_calls=1)
valid_ds = valid_ds.batch(BATCH_SIZE).cache('/home/paolo/ComputerSync/Università/ProgettoAgeEstimation/AgeEstimation/valid').prefetch(AUTOTUNE)



# **MAE METRICS**

In [ ]:
import tensorflow.keras.backend as K

def mae(y_true,y_pred):
  return K.mean(K.abs(K.argmax(y_true,-1)-K.argmax(y_pred,-1)),0)


# **DEFINIZIONE MODELLO**

In [ ]:
!pip install keras_applications
!pip install image-classifiers==1.0.0b1
from classification_models.keras import Classifiers
np.random.seed(23)
os.environ['PYTHONHASHSEED'] = '5'
random.seed(666)
tf.random.set_seed(666)
tf.compat.v1.reset_default_graph()
tf.keras.backend.clear_session()


EPOCHS = 50
size = (100,100,3)


ResNet18, preprocess_input = Classifiers.get('resnet18')
model = ResNet18(size, weights=None,classes=NUM_CLASSES,classifier_activation="softmax")

# **CALLBACKS**

In [ ]:
# RIDUCE IL LEARNING RATE SE LA LOSS SI FERMA
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=5,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0
)

#FERMA IL TRAINING SE LA VALIDATION LOSS RESTA FERMA PER TROPPE EPOCHE
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience=10,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

#SALVA IL MODELLO DOPO OGNI EPOCA
model_check = tf.keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/temp_model.h5",
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode="auto"
    )


callbacks = [model_check,reduce_lr,early_stop]


# **FIT**

In [ ]:
from multiprocessing import cpu_count
import pandas as pd

epochs = {
    0:50,
    1:100
}


for i in range(2):
  EPOCHS = epochs[i]

  model_check = tf.keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/model%s.h5" % (i),
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode="auto"
    )

  if i == 0:
    opt = DemonAdam(637*EPOCHS)
    callbacks = [model_check,reduce_lr]
  else:
    opt = DemonSGD(EPOCHS*637,learning_rate=0.01,momentum=0.9,nesterov=True)
    callbacks = [model_check,reduce_lr,lr_scheduler]
  
  

  model.compile(
      optimizer=opt,
      loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.15),
      metrics=[mae]
  )



  history = model.fit(
      train_ds,
      epochs=EPOCHS,
      batch_size=None,
      validation_data=valid_ds,
      callbacks=callbacks,
      class_weight=class_weights,
      use_multiprocessing=True,
      shuffle=False,
      workers=cpu_count(),
      verbose=1,
      max_queue_size=BATCH_SIZE,
      initial_epoch=0
  )


  # convert the history.history dict to a pandas DataFrame:     
  hist_df = pd.DataFrame(history.history) 
  PATH = '/content/drive/MyDrive/model%s.csv' % (EPOCHS)
  hist_csv_file = PATH
  with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)



# **SALVA IL MODELLO SU FILE**

In [ ]:
model.save('/content/drive/MyDrive/model.h5')

# ** EVALUATION SUL VALIDATION SET E MATRICE DI CONFUSIONE**

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

y_true = []
ds = valid_ds.unbatch()
for el in ds:
    y_true.append(np.argmax(el[1]))

print("Evaluation")
print(model.evaluate(valid_ds))

y_pred = model.predict(valid_ds)
argmax = np.argmax(y_pred,-1)
conf_mat = confusion_matrix(y_true,argmax)
sns.heatmap(conf_mat, annot=False,cmap=plt.cm.Reds)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

